In [49]:
%%writefile cmpsc472Project2.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/time.h>
#include <sys/resource.h>
#include <time.h>
#include <stdbool.h>

#define MAX_COLUMNS 40

/*
TODO:
- add global pool of resources to fight fires. This should be an int with an arbritrary value, e.g. 30 resources
- add fire fighting resources throughout the landscape
- determine whether to have multiple processes or multiple threads deal with the wildfires - Note: have parent process act as dispatch, have a child process search for wildfires, have
multiple threads from that child process fight the wildfires
- implement inter-process communication
- use semaphores/mutexes to lock critical sections
- have logic for extinguishing wildfires, add resources back to global resource pool when a fire is extinguished
*/

int global_resources = 10;

struct Landscape {
    int rows;
    int columns;
    int (*grid)[MAX_COLUMNS];
};

void printLandscape(struct Landscape *landscape);

int freeResources[global_resources][2];

struct Fire {
  int row;
  int col;
  int requiredResources;
};


// Function to set a random position in the array to 1, which represents a wildfire.
struct Fire setFire(struct Landscape *landscape) {
  struct Fire fire;
  // Seed the random number generator
  srand(time(NULL));
  int numOfRows = landscape -> rows;
  int numOfColumns= landscape -> columns;
  do
  {
  // Randomly select a row
  fire.row = rand() % (numOfRows - 0 + 1) + 0;
  // Randomly select a column
  fire.col = rand() % (numOfColumns - 0 + 1) + 0;
  }while (landscape -> grid[fire.row][fire.col] != 0);

  // Set the value at the selected row and column to 1
  landscape -> grid[fire.row][fire.col] = 1;
  printf("new fire at position (%d,%d)\n", fire.row, fire.col);
  // Randomly assign a required number of resources to extinguish the fire.
  fire.requiredResources = rand() % (global_resources - 0 + 1) + 0;
  global_resources -= fire.requiredResources;
  printf("required resources: %d\n", fire.requiredResources);
  return fire;
}

void placeResources(struct Landscape *landscape)
{

  // Seed the random number generator
  srand(time(NULL));

  int numOfRows = landscape -> rows;
  int numOfColumns= landscape -> columns;

  int resourceRow = 0;
  int resourceCol = 0;

  for (int i = 0; i < global_resources; i++)
  {

    do
    {

        resourceRow = rand() % (numOfRows) + 0;
        resourceCol = rand() % (numOfColumns - 0) + 0;

    }while (landscape -> grid[resourceRow][resourceCol] != 0);

    landscape -> grid[resourceRow][resourceCol] = 7;
    freeResources[i][0] = resourceRow;
    freeResources[i][1] = resourceCol;
    printf("new resource at position (%d,%d)\n", resourceRow, resourceCol);
  }

}

// Prints out the entire landscape array.
void printLandscape(struct Landscape *landscape) {
  for (int i = 0; i < landscape->rows; i++) {
    for (int j = 0; j < landscape->columns; j++) {
      if (landscape -> grid[i][j] == 1) // fire
      {
        printf("\033[1;91m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      else if (landscape -> grid[i][j] == 0) // ground
      {
        printf("\033[1;32m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      else if (landscape -> grid[i][j] == 5) // 911 Center
      {
        printf("\033[1;94m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      else if (landscape -> grid[i][j] == 7) // Firefighting resource
      {
        printf("\033[1;93m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      printf("%d ", landscape->grid[i][j]);
    }
    // Print a newline at the end of each row.
    printf("\n");
  }
}

void spiralScanForFreeResource(struct Landscape *landscape, int row, int col, int *result)
{

    int bound = 0;

    if ((landscape -> rows) - row > row)
    {

        bound = (landscape -> rows) - row;

    }
    else
    {

        bound = row;

    }
    for(int i = 1; i < bound+1; i++)
    {

        for (int j = row - i; j <= row + i; j++)
        {

            for (int k = col - i; k <= col + i; k++)
            {

                if (j >= landscape -> rows || j < 0 || k >= landscape -> columns || k < 0)
                {
                    continue;
                }
                else if (j > row - i && j < row + i && k > col - i &&  k < col + i)
                {
                    continue;
                }
                else
                {
                     if (landscape -> grid[j][k] == 7)
                     {

                         result[0] = j;
                         result[1] = k;
                         return;

                     }

                }

            }

        }

    }

}


int main() {
  // Create pipes for IPC
  int pipe_fd[2];
  if (pipe(pipe_fd) == -1) {
    perror("pipe");
    exit(EXIT_FAILURE);
  }

  struct Landscape *landscape = malloc(sizeof(struct Landscape));

  if (landscape == NULL) {
      fprintf(stderr, "Memory allocation failed\n");
      return 1;
  }

  // Define number of rows
  landscape -> rows = 40;
  // Define number of columns
  landscape -> columns = 40;
  // Create array to represent the landscape
  int landscapeGrid[landscape -> rows][landscape -> columns];
  // setting the array to the landscape struct
  landscape -> grid = landscapeGrid;
  int totalGridRows = landscape -> rows;
  int totalGridColumns = landscape -> columns;
  // Fill array with 0s
  for (int i = 0; i < totalGridRows; i++)
  {
      for (int j = 0; j < totalGridColumns; j++)
      {
        if (i == totalGridRows / 2)
        {
            landscape -> grid[i][j] = 5;
            if (j == totalGridColumns / 2)
            {
              landscape -> grid[i][j] = 5;
              continue;
            }
        }
        landscape -> grid[i][j] = 0;
      }
  }

  placeResources(landscape);

  // Number of fires to be set.
  int numOfFires = 5;
  // Amount of time to wait between setting fires.
  int waitTime;
  // Create array of Fire structs to store info on each fire set.
  struct Fire fires[numOfFires];
  // For loop to set the fires.
  for (int j = 0; j < numOfFires; j++) {
    // Stores attributes (row, col, number of required resources) about the fire in jth element of Fire struct array.
    fires[j] = setFire(landscape);
    printLandscape(landscape);
    printf("\n");
    // Seed random number generator
    srand(time(NULL));
    // Wait between 1 and 10 seconds before setting another fire.
    waitTime = rand() % (5 - 1 + 1) + 1;
    // Wait a random amount of time before setting another fire.
    sleep(waitTime);
  }

    printf("\n");
    int spiralScanResults[2] = {0};
    spiralScanForFreeResource(landscape, 39, 0, spiralScanResults);
    for (int i = 0; i < sizeof(spiralScanResults)/sizeof(spiralScanResults[0]); i++)
    {
        printf("%d ", spiralScanResults[i]);
    }
    printf("\n");

/*
Note: How to use a child and parent process. Have the child process set a boolean flag value equal to true (1) after it completes searching through the landscape. Then have the child process sleep.
While the child is searching, it should communicate the coordinates, and number of resources to the parent process. While the child process is sleeping, call wait().
Maybe have the parent process execute a while loop. While completeSearch = false, use continue statement, unless a fire is being communicated via IPC. Once the boolean flag is true, exit the loop, call wait()
*/


  // Now that all fires are set, create a child process to search for fires
  pid_t child_pid = fork();
  if (child_pid == 0) {
    // Counter to iterate through fires array
    int k = 0;
    // Search for fires in the landscape.
    // If one is found, use IPC to communicate that with the parent process.
    // Close read end of pipe
    close(pipe_fd[0]);
    // Search landscape for fires
    for (int i = 0; i < landscape -> rows; i++) {
      for (int j = 0; j < landscape ->columns; j++) {
        if (landscape -> grid[i][j] == 1) {
          // Communicate these coordinates to parent process
          write(pipe_fd[1], &i, sizeof(i));
          write(pipe_fd[1], &j, sizeof(j));

          // Communicate number of resources to parent process
          write(pipe_fd[1], &fires[k].requiredResources, sizeof(fires[k].requiredResources));
          // Increment counter to get required resources for next fire
          k++;
        }
      }
    }
    // After child finishes search of landscape, notify the parent
    int complete = -1;
    write(pipe_fd[1], &complete, sizeof(-1));
    write(pipe_fd[1], &complete, sizeof(-1));

    // Close write end of pipe
    close(pipe_fd[1]);

  } else if (child_pid > 0) {
    // Parent process
    // Variables to store the row, column, requiredResources passed from child process
    int r, c, rr;
    // Close write end of pipe
    close(pipe_fd[1]);

    // While child process searches for fires, listen for IPC from child process
    while (true) {
      read(pipe_fd[0], &r, sizeof(r));
      if (r != -1)
      {
        printf("Fire row: %d\n", r);
      }
      read(pipe_fd[0], &c, sizeof(c));
      if (c != -1)
      {
        printf("Fire col: %d\n", c);
        read(pipe_fd[0], &rr, sizeof(rr));
        printf("Num of resources: %d\n", rr);
      }

      printf("\n");

      if (r == -1 && c == -1) {
        // Child process has finished searching for fires
        break;
      }
    }
    // Wait to ensure child process has finished executing.
    wait(NULL);

  } else {
    perror("Failed to create child process.");
    exit(EXIT_FAILURE);
  }

  return 0;
}

Overwriting cmpsc472Project2.c


In [50]:
%%shell
gcc cmpsc472Project2.c -o cmpsc472Project2
./cmpsc472Project2

new resource at position (29,29)
new resource at position (39,35)
new resource at position (18,6)
new resource at position (28,11)
new resource at position (8,7)
new resource at position (37,36)
new resource at position (38,18)
new resource at position (10,36)
new resource at position (1,16)
new resource at position (11,0)
new fire at position (26,5)
required resources: 4
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 